In [35]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Importar otras librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------

import csv

# Configuraciones
# -----------------------------------------------------------------------

pd.set_option('display.max_columns', None) # Muestra todas las columnas de los DataFrame
pd.set_option('display.width', 1000) # Ajusta el ancho de la consola para evitar truncamientos

1. Creamos conexión con SQL.

In [36]:
config = {
  'user': 'root',
  'password': 'AlumnaAdalab',
  'host': '127.0.0.1',
  'raise_on_warnings': True
}

try:
  cnx = mysql.connector.connect(**config)
  print("Conexión exitosa")

except mysql.connector.Error as err:

  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Hay un problema con tu nombre de usuario o contraseña")
  
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("La base de datos no existe")
  
  else:
    print(err)

# Tratamos de crear la conexión con SQL. Usamos try/except para manejar los posibles errores.

config = {
  'user': 'root',
  'password': 'AlumnaAdalab',
  'host': '127.0.0.1',
  'raise_on_warnings': True
}

try:
  cnx = mysql.connector.connect(**config)

except mysql.connector.Error as err:

  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Hay un problema con tu nombre de usuario o contraseña")
  
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("La base de datos no existe")
  
  else:
    print(err)

Conexión exitosa


In [37]:
df = pd.read_csv('df_limpio_nulos_gestionados.csv', index_col=0)
df.head()

# df.dtypes

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,yearincome,monthlyincome,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,roledepartament,remotework
0,51.0,No,rarely,684.0,unknown,6,3,unknown,162.0,1,M,51.0,3,5,research director,3,unknown,6462,7,no,13,3.0,3,40.0,234444.000000,19537.000000,0,11.318934,5,3.00,20,15,15,1972,unknown,Si
1,52.0,No,rarely,699.0,unknown,1,4,Life Sciences,259.0,3,M,65.0,2,5,manager,3,unknown,5678,0,unknown,14,3.0,1,40.0,239988.000000,19999.000000,1,34.000000,5,3.00,33,11,9,1971,unknown,Si
2,42.0,No,rarely,532.0,Research & Development,4,2,Technical Degree,319.0,3,M,58.0,3,5,manager,4,married,4933,1,no,11,3.0,4,40.0,230784.000000,19232.000000,0,22.000000,3,2.76,22,11,15,1981,manager - research & development,Si
3,47.0,No,rarely,359.0,unknown,2,4,Medical,unknown,1,F,82.0,3,4,research director,3,married,26703,3,unknown,19,3.0,2,40.0,206028.000000,17169.000000,2,11.318934,2,2.76,20,5,6,1976,unknown,No
4,46.0,No,rarely,1319.0,unknown,3,3,Technical Degree,unknown,1,F,45.0,4,4,sales executive,1,divorced,7739,2,no,12,3.0,4,40.0,77732.233463,6477.686122,1,11.318934,5,3.00,19,2,8,1977,unknown,No


2. Creamos nuestra base de datos: keeptalent

In [39]:
# Creamos un cursor para poder ejecutar nuestras consultas 

mycursor = cnx.cursor()

# Creamos nuestra base de datos (schema)

try:
    mycursor.execute("CREATE DATABASE keeptalent")
    print(mycursor)

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)



CMySQLCursor: CREATE DATABASE keeptalent


3. Creamos una función para conectarnos con cualquier base de datos.

In [40]:

def establecer_conexion(database):

    # Esta función conecta con la base de datos que se le indica por parámetro y devuelve el conector creado

    config = {
    'user': 'root',
    'password': 'AlumnaAdalab',
    'host': '127.0.0.1',
    'database' : f'{database}',
    'raise_on_warnings': True
    }

    try:
        cnx = mysql.connector.connect(**config)
        print(f"Conexión establecida con {database}")

    except mysql.connector.Error as err:

        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Hay un problema con tu nombre de usuario o contraseña")
        
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("La base de datos no existe")
        
        else:
            print(err)
    
    return cnx



Creamos la tabla empleado 

In [41]:

cnx = establecer_conexion('keeptalent')

# Dejo los INT sin indicar el ancho (salía un warning de deprecated)
# No incluimos id_puesto porque la tabla no está creada todavía y no podemos relacionarla. 
# Y quizás no es imprescindible

query_crear_tabla_empleado = """CREATE TABLE empleado (

    id_empleado INT PRIMARY KEY AUTO_INCREMENT,
    age FLOAT,
    attrition VARCHAR(10),
    distancefromhome INT,        
    education INT, 
    educationfield VARCHAR(10),
    employeenumber VARCHAR(10),
    gender VARCHAR(5),
    maritalstatus VARCHAR(10), 
    datebirth INT,
    totalworkingyears FLOAT,
    yearsatcompany INT,
    numcompaniesworked INT)                  
    """ 

mycursor = cnx.cursor()

try:
    mycursor.execute(query_crear_tabla_empleado)
    print(mycursor)
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla.")
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)


                                    

Conexión establecida con keeptalent
CMySQLCursor: CREATE TABLE empleado (

    id_empleado..


Creamos la tabla puesto.

In [42]:

cnx = establecer_conexion('keeptalent')

query_crear_tabla_puesto = """CREATE TABLE puesto (

    id_puesto INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    id_empleado INT,
    joblevel INT,
    jobrole VARCHAR(100), 
    roledepartament VARCHAR(100),
    department VARCHAR(100),
    businesstravel VARCHAR(20),
    overtime VARCHAR(20),
    remotework VARCHAR(10),
    yearssincelastpromotion INT,
    yearswithcurrmanager INT, 
    trainingtimeslastyear FLOAT, 
    standardhours FLOAT,
    FOREIGN KEY (id_empleado) REFERENCES empleado (id_empleado))
    """ 

mycursor = cnx.cursor()


try:
    mycursor.execute(query_crear_tabla_puesto)
    print(mycursor)
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla.")
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

                                    

Conexión establecida con keeptalent
CMySQLCursor: CREATE TABLE puesto (

    id_puesto INT..


Creamos la tabla valoraciones 

In [43]:

cnx = establecer_conexion('keeptalent')

query_crear_tabla_puesto = """CREATE TABLE valoraciones (

    id_puesto INT,
    id_empleado INT,
    environmentsatisfaction INT,
    jobinvolvement INT,
    jobsatisfaction INT,
    relationshipsatisfaction INT,
    trainingtimeslastyear INT, 
    worklifebalance FLOAT,
    percentsalaryhike INT,
    PRIMARY KEY (id_empleado, id_puesto),
    FOREIGN KEY (id_empleado) REFERENCES empleado(id_empleado),
    FOREIGN KEY (id_puesto) REFERENCES puesto(id_puesto) 
    )
    """ 

mycursor = cnx.cursor()


try:
    mycursor.execute(query_crear_tabla_puesto)
    print(mycursor)
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla.")
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

                                    

Conexión establecida con keeptalent
CMySQLCursor: CREATE TABLE valoraciones (

    id_pues..


Y finalmente cerramos conexión

In [ ]:
cnx.close() 